# Pandas: Work with lines! — SOLUTION

This notebook completes the homework using the provided CSV files.

## Step 1 — Loading data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
PATH_DIR = "/content/drive/MyDrive/Colab Notebooks"

In [9]:
import os
import pandas as pd

In [18]:
def load_data(filename: str, drop_unnamed=True):
    if not os.path.exists(filename):
        print(f"{filename} does not exist")
        return None
    try:
        df = pd.read_csv(filename)
    except Exception:
        df = pd.read_csv(filename, sep=';')
    if drop_unnamed and "Unnamed: 0" in df.columns:
        df = df.drop(columns=["Unnamed: 0"])
    return df


In [19]:
missing_path = os.path.join(PATH_DIR, "SocialNetworkAds.csv")
data = load_data(missing_path)

/content/drive/MyDrive/Colab Notebooks/SocialNetworkAds.csv does not exist


In [20]:
print(data)

None


In [21]:
str_csv_name = "dirty_str_house_prices.csv"
filename = os.path.join(PATH_DIR, str_csv_name)
pd_data = load_data(filename)

In [25]:
if pd_data is not None:
    pd.set_option("display.max_columns", None)
    pd.set_option("display.width", 200)

    print("\nСписок столбцов:")
    print(pd_data.columns.values.tolist())

    print("\nПервые 5 строк:")
    print(pd_data.head())

    print("\nПоследние 5 строк:")
    print(pd_data.tail())
else:
    print("\nНе удалось загрузить датасет — проверь путь и имя файла.")



Список столбцов:
['Id', 'MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition']

Первые 5 строк:
   Id MSZoning  Street LotShape LandContour Utilities LotConfig LandSlope Neighborhood Condition1 Condition2 BldgType HouseStyle  RoofStyle     RoofMatl    Exterior1st Exterior2nd MasVnrType  \
0   1      rl:   PAVE.     ;REg       \tlvl   AllPub.   InsidE:      gtl      COLLGCR<      norm;      NORM.     1fam   ?2story?   GABLE...     compshg!  ...Vinylsd...    Vinylsd.   Brkface>   
1   2    ...RL   <PAVE    reg\t        Lvl;  AllP

## Step 2 — Group counts for categorical columns

In [31]:

import os
import pandas as pd

try:
    pd_data
except NameError:
    PATH_DIR = "/content/drive/MyDrive/data"
    filename = os.path.join(PATH_DIR, "dirty_str_house_prices.csv")

    def load_data(filename: str):
        if not os.path.exists(filename):
            print(f"{filename} does not exist")
            return None
        try:
            return pd.read_csv(filename)
        except Exception:
            return pd.read_csv(filename, sep=';')
    pd_data = load_data(filename)

def count_by_group(data, column=""):
    if data is None:
        return {}
    if column and column in data.columns:
        return data[column].value_counts()
    obj_cols = [c for c in data.columns if data[c].dtype == 'object']
    result = {}
    for c in obj_cols:
        result[c] = data[c].value_counts()
    return result

result = count_by_group(data=pd_data, column="")
print(type(result))
print(result)

print("\n\n=== Пример: распределение значений в колонке 'LotShape' ===")
result_lotshape = count_by_group(data=pd_data, column="LotShape")
print(type(result_lotshape))
print(result_lotshape)



<class 'dict'>
{'MSZoning': MSZoning
RL      42
RL<     30
RL.     24
>RL     22
RL!     21
        ..
>RL!     1
<Rm      1
:RM!     1
<rm<     1
<Rl>     1
Name: count, Length: 410, dtype: int64, 'Street': Street
PAVE        22
Pave        20
pave        20
>pave       19
Pave...     19
            ..
;pave!       1
...Pave      1
;PaVe        1
 PAvE        1
...PAvE>     1
Name: count, Length: 428, dtype: int64, 'LotShape': LotShape
REG          18
Reg          17
 IR1         14
Reg<         14
!REG         14
             ..
 Ir1...       1
;reg.         1
ir2\t         1
?reg...       1
...REG...     1
Name: count, Length: 480, dtype: int64, 'LandContour': LandContour
Lvl        27
LVL        25
lvl        22
LVL.       19
>Lvl       18
           ..
...LvL      1
\tLvl<      1
.Low\t      1
LOw\t       1
Hls.        1
Name: count, Length: 456, dtype: int64, 'Utilities': Utilities
allpub      29
ALLPUB      23
ALLPUB      19
ALLPUB;     19
Allpub:     17
            ..
AlLPUB;  

## Step 3 — Clean the string-based dataset to match the clean schema

In [33]:

import os
import re
import numpy as np
import pandas as pd

try:
    PATH_DIR
except NameError:
    PATH_DIR = "/content/drive/MyDrive/data"

try:
    load_data
except NameError:
    def load_data(filename: str):
        if not os.path.exists(filename):
            print(f"{filename} does not exist")
            return None
        try:
            return pd.read_csv(filename)
        except Exception:
            return pd.read_csv(filename, sep=';')


dirty_path = os.path.join(PATH_DIR, "dirty_str_house_prices.csv")
pd_dirty_raw = load_data(dirty_path)

clean_path = os.path.join(PATH_DIR, "clean_house_prices.csv")
pd_clean_ref = load_data(clean_path)

if pd_dirty_raw is not None and "Unnamed: 0" in pd_dirty_raw.columns:
    pd_dirty_raw = pd_dirty_raw.drop(columns=["Unnamed: 0"])


def _basic_clean_str(x: object) -> object:
    if pd.isna(x):
        return x
    s = str(x)
    s = s.replace('\t', ' ').replace('\n', ' ')
    s = s.strip(" .;:,-_/\\\t\n")
    s = re.sub(r"\s+", " ", s)
    return s

def _to_numeric_safely(series: pd.Series) -> pd.Series:

    raw = series.astype(str)
    cleaned = raw.str.replace(r"[^0-9\-\.,]", "", regex=True).str.replace(",", "", regex=False)
    nums = pd.to_numeric(cleaned, errors="coerce")
    return nums

def _norm_key(v: object) -> str:
    v = str(v)
    v = v.lower()
    v = re.sub(r"[^a-z0-9]+", "", v)
    return v

def dirty_data(data: pd.DataFrame, ref: pd.DataFrame) -> pd.DataFrame:
    if data is None or ref is None:
        return data

    df = data.copy()


    for c in df.columns:
        if df[c].dtype == "object":
            df[c] = df[c].map(_basic_clean_str)


    if "Id" in df.columns:
        df = df.set_index("Id", drop=True)
    if "Id" in ref.columns:
        ref = ref.set_index("Id", drop=True)

    out = df.copy()
    for c in out.columns:
        if c in ref.columns:
            if pd.api.types.is_numeric_dtype(ref[c]):
                out[c] = _to_numeric_safely(out[c])
            else:
                ref_vals = ref[c].dropna().astype(str).unique()
                ref_map = {_norm_key(rv): rv for rv in ref_vals}

                def map_to_ref(val):
                    if pd.isna(val):
                        return val
                    key = _norm_key(val)
                    return ref_map.get(key, str(val).strip().title())
                out[c] = out[c].apply(map_to_ref)
        else:

            pass


    common_cols = [c for c in ref.columns if c in out.columns]
    out = out[common_cols].copy()


    out = out.reset_index()
    return out

pd_dirty_cleaned = dirty_data(pd_dirty_raw, pd_clean_ref)

print("До очистки (грязный) — первые 5 строк:\n", pd_dirty_raw.head(), "\n")
print("После очистки — первые 5 строк:\n", pd_dirty_cleaned.head())


До очистки (грязный) — первые 5 строк:
    Id MSZoning  Street LotShape LandContour Utilities LotConfig LandSlope Neighborhood Condition1 Condition2 BldgType HouseStyle  RoofStyle     RoofMatl    Exterior1st Exterior2nd MasVnrType  \
0   1      rl:   PAVE.     ;REg       \tlvl   AllPub.   InsidE:      gtl      COLLGCR<      norm;      NORM.     1fam   ?2story?   GABLE...     compshg!  ...Vinylsd...    Vinylsd.   Brkface>   
1   2    ...RL   <PAVE    reg\t        Lvl;  AllPUb\t      FR2       Gtl<    veenker\t    ?FEedr.      ;norm    1FAM:    :1story  .GABLE...     CoMPSHG>        METALSD    metalsd      ?NOnE.   
2   3     ;Rl    Pave      ir1?        LVL!   Allpub!  ?inside     ...Gtl     ?COLLGCR      !Norm     .norm>    !1fam    .2story      Gable  ...COMPSHG:       vinylsd.    ?Vinylsd   !brkface   
3   4       rl   pave.     ;IR1        .Lvl  ;ALLPUB;   >Corner      ;GTL     .CRAWFOR       NORm      NORM:    1FAM>    2STORY<   \tgable.    !Compshg<       WD SDNG:    Wd shng;     

## Step 4 — Compare cleaned dirty data with the clean data

In [7]:
from google.colab import drive
drive.mount('/content/drive')

import os, re, glob
import numpy as np
import pandas as pd

CLEAN_CANDIDATES = ["clean_house_prices.csv", "num_house_prices.csv"]
DIRTY_CANDIDATES = ["dirty_str_house_prices.csv", "str_house_prices.csv"]

def find_in_drive(names):
    for name in names:
        matches = glob.glob(f"/content/drive/MyDrive/**/{name}", recursive=True)
        if matches:
            print(f"Found {name}: {matches[0]}")
            return matches[0]
    return None

clean_path = find_in_drive(CLEAN_CANDIDATES)
dirty_path = find_in_drive(DIRTY_CANDIDATES)

if clean_path is None or dirty_path is None:
    raise FileNotFoundError(
        "Не удалось найти один из файлов в Google Drive. "
        "Проверь точные имена файлов или положи их в MyDrive. "
        f"Искали: clean={CLEAN_CANDIDATES}, dirty={DIRTY_CANDIDATES}"
    )

def load_data(filename: str):
    if not os.path.exists(filename):
        print(f"{filename} does not exist")
        return None
    try:
        return pd.read_csv(filename)
    except Exception:
        return pd.read_csv(filename, sep=';')

pd_clean_ref = load_data(clean_path)
pd_dirty_raw = load_data(dirty_path)

if pd_dirty_raw is not None and "Unnamed: 0" in pd_dirty_raw.columns:
    pd_dirty_raw = pd_dirty_raw.drop(columns=["Unnamed: 0"])

def _basic_clean_str(x):
    if pd.isna(x): return x
    s = str(x).replace('\t',' ').replace('\n',' ')
    s = s.strip(" .;:,-_/\\\t\n")
    s = re.sub(r"\s+", " ", s)
    return s

def _to_numeric_safely(series: pd.Series) -> pd.Series:
    raw = series.astype(str)
    cleaned = raw.str.replace(r"[^0-9\-\.,]", "", regex=True).str.replace(",", "", regex=False)
    return pd.to_numeric(cleaned, errors="coerce")

def _norm_key(v):
    if pd.isna(v): return ""
    return re.sub(r"[^a-z0-9]+", "", str(v).lower())

def dirty_data(data: pd.DataFrame, ref: pd.DataFrame) -> pd.DataFrame:
    if data is None or ref is None:
        return data
    df = data.copy()

    for c in df.columns:
        if df[c].dtype == "object":
            df[c] = df[c].map(_basic_clean_str)

    if "Id" in df.columns: df = df.set_index("Id", drop=True)
    if "Id" in ref.columns: ref = ref.set_index("Id", drop=True)

    out = df.copy()
    for c in out.columns:
        if c in ref.columns:
            if pd.api.types.is_numeric_dtype(ref[c]):
                out[c] = _to_numeric_safely(out[c])
            else:
                ref_vals = ref[c].dropna().astype(str).str.strip().unique()
                ref_map = {_norm_key(rv): rv for rv in ref_vals}
                def map_to_ref(val):
                    if pd.isna(val): return val
                    key = _norm_key(val)
                    return ref_map.get(key, str(val).strip())
                out[c] = out[c].apply(map_to_ref)

    out = out[[c for c in ref.columns if c in out.columns]].copy().reset_index()
    return out

pd_dirty_cleaned = dirty_data(pd_dirty_raw, pd_clean_ref)

L = pd_clean_ref.set_index('Id')
R = pd_dirty_cleaned.set_index('Id')

common_cols = [c for c in L.columns if c in R.columns]
idx = L.index.intersection(R.index)
L = L.loc[idx, common_cols].copy()
R = R.loc[idx, common_cols].copy()


for c in common_cols:
    if pd.api.types.is_numeric_dtype(L[c]) or pd.api.types.is_numeric_dtype(R[c]):
        L[c] = pd.to_numeric(L[c], errors='coerce')
        R[c] = pd.to_numeric(R[c], errors='coerce')
    else:
        L[c] = L[c].astype(str).str.strip()
        R[c] = R[c].astype(str).str.strip()


eq_all = (L.eq(R)) | (L.isna() & R.isna())
overall_pct = 100.0 * eq_all.stack().mean()
print(f"Overall (%) same: {overall_pct:.6f}")

USE_COMMA_DECIMAL = True
print("\n(%) same")
for c in common_cols:
    col_eq = (L[c].eq(R[c])) | (L[c].isna() & R[c].isna())
    pct = 100.0 * float(col_eq.mean())
    val = f"{pct:.6f}"
    if USE_COMMA_DECIMAL:
        val = val.replace('.', ',')
    print(f"{c:<14} {val}")


Mounted at /content/drive
Found clean_house_prices.csv: /content/drive/MyDrive/Colab Notebooks/clean_house_prices.csv
Found dirty_str_house_prices.csv: /content/drive/MyDrive/Colab Notebooks/dirty_str_house_prices.csv
Overall (%) same: 98.465896

(%) same
MSZoning       100,000000
Street         99,925262
LotShape       99,925262
LandContour    99,925262
Utilities      99,925262
LotConfig      99,925262
LandSlope      99,925262
Neighborhood   99,925262
Condition1     100,000000
Condition2     99,925262
BldgType       99,925262
HouseStyle     99,925262
RoofStyle      99,925262
RoofMatl       99,925262
Exterior1st    99,925262
Exterior2nd    99,925262
MasVnrType     44,245142
ExterQual      100,000000
ExterCond      99,925262
Foundation     99,925262
BsmtQual       99,925262
BsmtCond       99,925262
BsmtExposure   99,925262
BsmtFinType1   99,925262
BsmtFinType2   99,925262
Heating        99,925262
HeatingQC      99,925262
CentralAir     99,925262
Electrical     99,925262
KitchenQual    9